# 1. Generating a SHACL shape from a list of mandatory properties

In [42]:
from jinja2 import Template

mandatory_properties = ['schema:featureList', 
                        'schema:applicationSubCategory', 
                        'edam:has_input', 
                        'edam:has_output']

shape_template = """
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix edam: <http://edamontology.org/> .

schema:ToolShape
    a sh:NodeShape ;
    sh:targetClass schema:SoftwareApplication ;
    
    {% for prop_name in properties %}
    sh:property [
        sh:path {{prop_name}} ;
        sh:minCount 1 ;
    ] ;
    {% endfor %}
    .
"""

template = Template(shape_template)
shape = template.render(properties=mandatory_properties)
print(shape)
g = ConjunctiveGraph()
g.parse(data = shape, format='turtle')
print(len(g))


@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix edam: <http://edamontology.org/> .

schema:ToolShape
    a sh:NodeShape ;
    sh:targetClass schema:SoftwareApplication ;
    
    
    sh:property [
        sh:path schema:featureList ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path schema:applicationSubCategory ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path edam:has_input ;
        sh:minCount 1 ;
    ] ;
    
    sh:property [
        sh:path edam:has_output ;
        sh:minCount 1 ;
    ] ;
    
    .
14


# 2. Defining test data to be validated

In [34]:
manual_shape_constraint = """
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

schema:ToolShape
    a sh:NodeShape ;
    sh:targetClass schema:SoftwareApplication ;
    sh:property [
        sh:path schema:featureList ;
        sh:minCount 1 ;
    ] ;
    sh:property [
        sh:path schema:applicationSubCategory ;
        sh:minCount 1 ;
    ] .
"""

data = """
@prefix biotools: <https://bio.tools/ontology/> .
@prefix bsc: <http://bioschemas.org/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix doi: <https://doi.org/> .
@prefix edam: <http://edamontology.org/> .
@prefix pmc: <https://www.ncbi.nlm.nih.gov/pmc/> .
@prefix pubmed: <https://www.ncbi.nlm.nih.gov/pubmed/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sc: <http://schema.org/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .


<https://bio.tools/PyBDA> a sc:SoftwareApplication ;
    sc:additionalType "Command-line tool" ;
    sc:applicationSubCategory edam:topic_3391 ;
    sc:citation <https://doi.org/10.1186/s12859-019-3087-8> ;
    sc:description "A command-line tool for analysis of big biological data sets for distributed HPC clusters." ;
    sc:featureList edam:operation_2939,
        edam:operation_2945,
        edam:operation_3432,
        edam:operation_3659 ;
    sc:license "GPL-3.0" ;
    sc:name "PyBDA" ;
    sc:url "https://pybda.readthedocs.io/en/latest/#" .
"""

# 3. Validating the shape on the graph data

In [48]:
from pyshacl import validate
from IPython.display import display, Markdown, Latex

r = validate(data_graph = data, 
             data_graph_format='turtle', 
             shacl_graph = shape, 
             shacl_graph_format = 'turtle', 
             ont_graph = None, 
             inference = 'rdfs', 
             abort_on_error = False, 
             meta_shacl = False, 
             debug = True)

conforms, results_graph, results_text = r

if not conforms:
    print(results_graph.serialize(format='turtle').decode())
else:
    print('Everything is fine')

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_output ]
	Focus Node: <https://bio.tools/PyBDA>
	Result Path: edam:has_output

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/PyBDA>
	Result Path: edam:has_input



@prefix dash: <http://datashapes.org/dash#> .
@prefix edam: <http://edamontology.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <https://bio.tools/PyBDA> ;
            sh:resultPath edam:has_output ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
            sh:sourceShape [ sh:minCount 1 ;
                    sh:path edam:has_output ] ],
        [ a sh:ValidationResult ;
            sh:focusNode <https://bio.tools/PyBDA> ;
            sh:resultPath edam:has_input ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstrai

# 4. Producing an explaination for the validation graph

In [47]:
report_query = """
    SELECT ?node ?path WHERE {
        ?v rdf:type sh:ValidationReport ;
           sh:result ?r .
        ?r sh:focusNode ?node ;
           sh:sourceShape ?s .
        ?s sh:path ?path .   
    }
"""

results = results_graph.query(report_query)
for r in results :
    display(Markdown('The tool `{}` should be fixed, it is missing information for field {}'.format(str(r['node']), str(r['path']))))
    #display(Markdown(' **It was produced in the context of** ' + str(r['st']) ))

The tool `https://bio.tools/PyBDA` should be fixed, it is missing information for field http://edamontology.org/has_input

The tool `https://bio.tools/PyBDA` should be fixed, it is missing information for field http://edamontology.org/has_output

# 5. Launching the validation on a large dataset ..

In [51]:
r = validate(data_graph = 'biotools.bioschema.ttl', 
             data_graph_format='turtle', 
             shacl_graph = shape, 
             shacl_graph_format = 'turtle', 
             ont_graph = None, 
             inference = 'rdfs', 
             abort_on_error = False, 
             meta_shacl = False, 
             debug = True)

conforms, results_graph, results_text = r

results = results_graph.query(report_query)
for r in results :
    display(Markdown('The tool {} should be fixed, it is missing information for field {}'.format(str(r['node']), str(r['path']))))

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/SWPepNovo>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/EnClaSC>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/GEO2R>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	S

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/BIRD>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/RetSynth>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/ZIKAVID-Zika_virus_infection_database>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinC

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/FTAlign>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/OptMAVEn-2.0>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/ChromDragoNN>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComp

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/DNA>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/verdedb>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/textttbrea>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Se

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/miComplete>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/MinIONTM>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:minCount Literal("1", datatype=xsd:integer) ; sh:path edam:has_input ]
	Focus Node: <https://bio.tools/decision-tree>
	Result Path: edam:has_input

Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComp

...

The tool https://bio.tools/IMPLEMENTATIONHowDe-SBT should be fixed, it is missing information for field http://edamontology.org/has_output

The tool https://bio.tools/Garnett should be fixed, it is missing information for field http://edamontology.org/has_output

The tool https://bio.tools/SeQuiLa-cov should be fixed, it is missing information for field http://edamontology.org/has_output

The tool https://bio.tools/SpydrPick should be fixed, it is missing information for field http://edamontology.org/has_input

The tool https://bio.tools/FICC-Seq should be fixed, it is missing information for field http://edamontology.org/has_input

# 6. Basic statistics

In [55]:
count_query = """
    SELECT (COUNT(?node) AS ?to_be_fixed_entries) WHERE {
        ?v rdf:type sh:ValidationReport ;
           sh:result ?r .
        ?r sh:focusNode ?node ;
           sh:sourceShape ?s .
        ?s sh:path ?path .   
    }
"""

results = results_graph.query(count_query)
for r in results :
    display(Markdown('**{}** tools should be fixed.'.format(str(r['to_be_fixed_entries']))))



**2215** tools should be fixed.